In [1]:
import unidecode
import string
import random
import math

import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
all_characters = string.printable
input_size = len(all_characters)
output_size = input_size
print(input_size)

100


In [3]:
n_steps = 2000
batch_size = 512
hidden_size = 100
n_layers = 2
learning_rate = 0.01
len_text = 200
print_every = 50

In [4]:
filename = '../Data/obama.txt'
data = unidecode.unidecode(open(filename).read())
len_data = len(data)

In [5]:
def char_to_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor

In [6]:
def batch_gen(length_text, batch_size):
    X = torch.LongTensor(batch_size, length_text)
    y = torch.LongTensor(batch_size, length_text)
    for i in range(batch_size):
        start_index = random.randint(0, len_data - length_text)
        end_index = start_index + length_text + 1
        text = data[start_index:end_index]
        X[i] = char_to_tensor(text[:-1])
        y[i] = char_to_tensor(text[1:])
    X = Variable(X)
    y = Variable(y)
    X = X.cuda()
    y = y.cuda()
    return X, y

In [7]:
class create_model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(create_model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))


In [9]:
decoder_model = create_model(
    input_size,
    hidden_size,
    output_size,
    n_layers=n_layers,
)

opt = torch.optim.Adam(decoder_model.parameters(), lr=learning_rate)
loss = nn.CrossEntropyLoss()
decoder_model.cuda()

create_model (
  (encoder): Embedding(100, 100)
  (rnn): GRU(100, 100, num_layers=2)
  (decoder): Linear (100 -> 100)
)

In [10]:
def generate_text(decoder, start='The', predict_len=100):
    hidden = decoder.init_hidden(1).cuda()
    prime_input = Variable(char_to_tensor(start).unsqueeze(0)).cuda()
    predicted = start

    for p in range(len(start) - 1):
        _, hidden = decoder(prime_input[:, p], hidden)
        
    x = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(x, hidden)
        output_dist = output.data.view(-1).div(0.8).exp()
        # Add some randomness
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        x = Variable(char_to_tensor(predicted_char).unsqueeze(0)).cuda()
    return predicted

In [11]:
loss_avg = 0

for i in range(n_steps):
    X, y = batch_gen(len_text, batch_size)
    hidden = decoder_model.init_hidden(batch_size).cuda()
    decoder_model.zero_grad()
    loss_total = 0

    for c in range(len_text):
        output, hidden = decoder_model(X[:,c], hidden)
        loss_total += loss(output.view(batch_size, -1), y[:,c])

    loss_total.backward()
    opt.step()
    loss_value = loss_total.data[0] / len_text
    
    loss_avg += loss_value

    if i % print_every == 0:
        print('Epoch {}: loss {}'.format(i, loss_avg))
        print(generate_text(decoder_model, 'The', 100), '\n')

Epoch 0: loss 4.601166687011719
Y{zHX2G!PDCP_k0@9vbrTy`y3t^	u/b:<Xz,Jhrw!g",C	%hz mWh

Epoch 50: loss 129.82943481445315
The incy to mind have of ture the that be pare hor whas menaring it therast perdertions the and in mane 

Epoch 100: loss 223.73026443481436
The secure, I unstorich to have in that cound this as cruration in the by mathout a party, have the bas 

Epoch 150: loss 304.88177886962853
There but and the memple polinizator eefforge deterated simus our government because of the problem an  

Epoch 200: loss 380.0532673645016
The finary and less securing on the time of your process the young for a hoiser of the some to complete 

Epoch 250: loss 451.83719955444315
There it is a country from health care for politics can going to move away against more and they is kno 

Epoch 300: loss 521.4999079895017
The people as the world that the standing a man warming-to they classing this simple the
 individual pr 



RuntimeError: inconsistent tensor size, expected tensor [200] and src [199] to have the same number of elements, but got 200 and 199 elements respectively at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/TH/generic/THTensorCopy.c:86